# ML4NLP1
## Starting Point for Exercise 1, part I

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part I.

One of the goals of this exercise is o make you acquainted with sklearn and related libraries like pandas and numpy. You will probably need to consult the documentation of those libraries:
- sklearn: [Documentation](https://scikit-learn.org/stable/user_guide.html)
- Pandas: [Documentation](https://pandas.pydata.org/docs/#)
- Numpy: [Documentation](https://numpy.org/doc/)

**Importing files to Google Colab:** If you have never used Colab or never uploaded a file to Colab, quickly skim over an introduction: [Introduction on medium](https://medium.com/@master_yi/importing-datasets-in-google-colab-c816fc654f97).

We're using the second method mentioned in the blogpost: (1) upload the four files `x_train.txt` and `y_train.txt`, `x_test.txt` and `y_test.txt` to a directory in Google Drive and (2) adjust the paths in the second cell to point to your uploaded files.

Then execute the first cell to give Colab permission to access the two files.

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

In [ ]:
# download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /content/x_train.txt
100% 64.1M/64.1M [00:01<00:00, 57.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /content/x_test.txt
100% 65.2M/65.2M [00:00<00:00, 77.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /content/y_train.txt
100% 480k/480k [00:00<00:00, 5.09MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /content/y_test.txt
100% 480k/480k [00:00<00:00, 5.10MB/s]


In [ ]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [ ]:
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})
# write train_df to csv with tab as separator
train_df.to_csv('train_df.csv', index=False, sep='\t')
# comibne x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

#combine both train_df and test_df into one dataframe for further data split according to 80/20 rule
total_df = pd.concat([train_df, test_df])
total_df.to_csv('file.csv', index=False, sep='\t')

In [ ]:
# get list of all labels
labels = train_df['label'].unique().tolist()
print(labels)

['est', 'swe', 'mai', 'oci', 'tha', 'orm', 'lim', 'guj', 'pnb', 'zea', 'krc', 'hat', 'pcd', 'tam', 'vie', 'pan', 'szl', 'ckb', 'fur', 'wuu', 'arz', 'ton', 'eus', 'map-bms', 'glk', 'nld', 'bod', 'jpn', 'arg', 'srd', 'ext', 'sin', 'kur', 'che', 'tuk', 'pag', 'tur', 'als', 'koi', 'lat', 'urd', 'tat', 'bxr', 'ind', 'kir', 'zh-yue', 'dan', 'por', 'fra', 'ori', 'nob', 'jbo', 'kok', 'amh', 'khm', 'hbs', 'slv', 'bos', 'tet', 'zho', 'kor', 'sah', 'rup', 'ast', 'wol', 'bul', 'gla', 'msa', 'crh', 'lug', 'sun', 'bre', 'mon', 'nep', 'ibo', 'cdo', 'asm', 'grn', 'hin', 'mar', 'lin', 'ile', 'lmo', 'mya', 'ilo', 'csb', 'tyv', 'gle', 'nan', 'jam', 'scn', 'be-tarask', 'diq', 'cor', 'fao', 'mlg', 'yid', 'sme', 'spa', 'kbd', 'udm', 'isl', 'ksh', 'san', 'aze', 'nap', 'dsb', 'pam', 'cym', 'srp', 'stq', 'tel', 'swa', 'vls', 'mzn', 'bel', 'lad', 'ina', 'ava', 'lao', 'min', 'ita', 'nds-nl', 'oss', 'kab', 'pus', 'fin', 'snd', 'kaa', 'fas', 'cbk', 'cat', 'nci', 'mhr', 'roa-tara', 'frp', 'ron', 'new', 'bar', 'ltg'

In [ ]:
# T: Have a quick peek at the training data, looking at a couple of texts from different languages. Do you notice anything that might be challenging for the classification?

In [ ]:
train_df.head(100)

,text,label
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha
...,...,...
95,"(fr) Hélène Scmitz; Nils Uddenberg, Linné : le...",lin
96,"Li fisic scienties es afini a altri scienties,...",ile
97,"Ewa Bożena Kopacz, nascet quam Ewa Bożena Lis,...",ile
98,Mombaroccio (Mombaròcc en Gal-Italich Marchigi...,lmo


Challenges:
1. Some languages are quite similar. e.g. ckb(index17) and glk(index24).
2. Some sentences have code-switching(index19), it can be hard to recognise the language of this sentence.

In [ ]:
# T: How many instances per label are there in the training and test set? Do you think this is a balanced dataset? Do you think the train/test split is appropriate? If not, please rearrange the data in a more appropriate way.

In [ ]:
train_df.groupby(['label']).count()

,text
label,
ace,500
afr,500
als,500
amh,500
ang,500
...,...
yid,500
yor,500
zea,500


In [ ]:
test_df.groupby(['label']).count()

,text
label,
ace,500
afr,500
als,500
amh,500
ang,500
...,...
yid,500
yor,500
zea,500


Every label in train/test data has 500 instances. It's a balanced dataset.
Both training set and test set have 235 rows, which means that the data is split into 50% training and 50% testing. I think this is not appropriate. I would split the data according to 80/20 rule.

In [ ]:
# T: Get a subset of the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)

In [ ]:
# choose 20 additional languages
# languages = ['eng', 'deu', 'nld', 'dan', 'swe', 'nob', 'lim', 'glv', 'egl', 'ind','eus', 'hat', 'est', 'oci', 'map_bms', 'arg', 'ext', 'tuk', 'pag', 'tur', 'als', 'ind', 'ast', 'wol', 'rup', 'lug']

In [ ]:
# Ranodomly choose 20 additional languages
# As a dataset composed of 27 languages would cause the session to crash due to ram limit.
# So I only use 7 languages as my dataset.
import random
random.seed(20)

languages = ['eng', 'deu', 'nld', 'dan', 'swe', 'nob', 'jpn']
#for i in range(20):
  #languages.append(random.choice(labels))
print(languages)

subset_df = total_df[total_df['label'].isin(languages)]
subset_df

['eng', 'deu', 'nld', 'dan', 'swe', 'nob', 'jpn']


,text,label
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
26,De spons behoort tot het geslacht Haliclona en...,nld
29,エノが行きがかりでバスに乗ってしまい、気分が悪くなった際に助けるが、今すぐバスを降りたいと運...,jpn
46,シャーリー・フィールドは、サン・ベルナルド・アベニュー沿い市民センターとR&Tマーティン高校...,jpn
52,Indtil 1545 havde flådecheferne kunnet hyre et...,dan
...,...,...
117156,Het gewone volk leeft in vrede en nog altijd w...,nld
117196,På midten av 1980-tallet spilte han sammen med...,nob
117202,"katholische Ortskirche St. Nikolaus (1955), be...",deu
117308,Mozilla Add-ons は、Mozilla Firefox、Mozilla Thun...,jpn


In [ ]:
text_df = subset_df['text']
label_df = subset_df['label']
text_df.shape, label_df.shape

((7000,), (7000,))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(text_df, label_df, test_size=0.20, random_state=42)


In [ ]:
print('Training set shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test set shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training set shape:  (5600,)
Training labels shape:  (5600,)
Test set shape:  (1400,)
Test labels shape:  (1400,)


Pipeline as a shortcut

In [ ]:
# Define the charater ngram vectorizers
char_ngram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 5))
# Define the word bigram vectorizer
word_ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 5))

# Combine the features
combined_features = FeatureUnion([
    ('char_ngram', char_ngram_vectorizer),
    ('word_ngram', word_ngram_vectorizer)
])

text_clf = Pipeline([
    ('features', combined_features),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression())
])

In [ ]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('char_ngram',
                                                 CountVectorizer(analyzer='char',
                                                                 ngram_range=(2,
                                                                              5))),
                                                ('word_ngram',
                                                 CountVectorizer(ngram_range=(2,
                                                                              5)))])),
                ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])

In [ ]:
scores = cross_val_score(text_clf, X_train, y_train, scoring='accuracy', cv=3)
scores

array([0.98875201, 0.98660953, 0.98874598])

LabelEncoder

In [ ]:
# T: With the following code, we wanted to encode the labels, however, our cat was walking on the keyboard and some of it got changed. Can you fix it?
# from sklearn.preprocessing import LabelEncoder
# le_fitted = LabelEncoder().fit(train_df['label'])
# y_train_dev, y_test = le_fitted.fit(train_df['label']), le_fitted.fit(test_df['label'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Fit the encoder on the training data
le_fitted = LabelEncoder().fit(train_df['label'])

# Transform the labels of train and test datasets
y_train_dev = le_fitted.transform(train_df['label'])
y_test = le_fitted.transform(test_df['label'])


Model selection

In [ ]:
# Model selection

from sklearn.model_selection import GridSearchCV

text_lore = Pipeline([
    ('vect', combined_features),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression())
])


param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [0.01, 0.1],
    'clf__solver': ['saga', 'sag'],
}


gs_svc = GridSearchCV(text_lore, param_grid, cv=2, verbose=1)
gs_svc.fit(X_train, y_train)


Fitting 2 folds for each of 8 candidates, totalling 16 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
4 fits failed out of a total of 16.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  F

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        FeatureUnion(transformer_list=[('char_ngram',
                                                                        CountVectorizer(analyzer='char',
                                                                                        ngram_range=(2,
                                                                                                     5))),
                                                                       ('word_ngram',
                                                                        CountVectorizer(ngram_range=(2,
                                                                                                     5)))])),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', LogisticRegression())]),
             param_grid={'clf__C': [0.01, 0.1], 'clf__penalty': ['l1', 'l2'],
                         'clf__solver': ['saga', 'sag']},
             verbose=1)

In [ ]:
svc_df = pd.DataFrame.from_dict(gs_svc.cv_results_)
svc_df.sort_values(by=["rank_test_score"])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,param_clf__penalty,param_clf__solver,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
7,11.939587,0.711554,3.631643,0.452191,0.1,l2,sag,"{'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__so...",0.980714,0.981071,0.980893,0.000179,1
6,14.825698,0.477359,3.153888,0.120293,0.1,l2,saga,"{'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__so...",0.980357,0.981429,0.980893,0.000536,2
3,17.654330,1.195998,3.658830,0.556872,0.01,l2,sag,"{'clf__C': 0.01, 'clf__penalty': 'l2', 'clf__s...",0.960714,0.970357,0.965536,0.004821,3
2,16.490607,0.668070,3.123000,0.035489,0.01,l2,saga,"{'clf__C': 0.01, 'clf__penalty': 'l2', 'clf__s...",0.957857,0.969286,0.963571,0.005714,4
4,54.837693,0.291681,3.134524,0.065623,0.1,l1,saga,"{'clf__C': 0.1, 'clf__penalty': 'l1', 'clf__so...",0.746786,0.753929,0.750357,0.003571,5
0,9.591295,0.673640,3.980155,0.109168,0.01,l1,saga,"{'clf__C': 0.01, 'clf__penalty': 'l1', 'clf__s...",0.143929,0.141071,0.142500,0.001429,6
1,6.977325,0.156735,0.000000,0.000000,0.01,l1,sag,"{'clf__C': 0.01, 'clf__penalty': 'l1', 'clf__s...",NaN,NaN,NaN,NaN,7
5,6.989137,0.533077,0.000000,0.000000,0.1,l1,sag,"{'clf__C': 0.1, 'clf__penalty': 'l1', 'clf__so...",NaN,NaN,NaN,NaN,7


In [ ]:
print("Best parameters:", gs_svc.best_params_)
best_model = gs_svc.best_estimator_

Best parameters: {'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__solver': 'sag'}


In [ ]:
best_model = Pipeline([
    ('vect', combined_features),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(C=0.1, penalty='l2', solver='sag'))
])

best_model.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 FeatureUnion(transformer_list=[('char_ngram',
                                                 CountVectorizer(analyzer='char',
                                                                 ngram_range=(2,
                                                                              5))),
                                                ('word_ngram',
                                                 CountVectorizer(ngram_range=(2,
                                                                              5)))])),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=0.1, solver='sag'))])

In [ ]:
from sklearn.metrics import accuracy_score

predictions = best_model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print('Accuracy:', accuracy)


Accuracy: 0.9764285714285714


In [ ]:
!pip install eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107719 sha256=f5762793f2ed1eaa7bb0beccd44bdbf4f20218cf189533c295fc64f6f8de7558
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [ ]:
languages = ['eng', 'swe', 'nob', 'jpn']
subset_df = total_df[total_df['label'].isin(languages)]

text_df = subset_df['text']
label_df = subset_df['label']
text_df.shape, label_df.shape

((4000,), (4000,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text_df, label_df, test_size=0.20, random_state=42)

In [ ]:
import eli5

# Top 10 character ngrams

# Creating a pipeline
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 5))
clf = LogisticRegression()

# Fit the vectorizer on the data
X_transformed = vectorizer.fit_transform(X_train)

# Fit the logistic regression classifier
clf.fit(X_transformed, y_train)

# Manually extracting feature names from CountVectorizer
feature_names = vectorizer.get_feature_names_out()

# Use eli5 to show weights, passing the feature names manually
eli5.show_weights(clf, vec=vectorizer, top=10, feature_names=feature_names)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Top 10 word ngrams

# Creating a pipeline
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 5))
clf = LogisticRegression()

# Fit the vectorizer on your data
X_transformed = vectorizer.fit_transform(X_train)

# Fit the logistic regression classifier
clf.fit(X_transformed, y_train)

# Manually extracting feature names from CountVectorizer
feature_names = vectorizer.get_feature_names_out()

# Now use eli5 to show weights, passing the feature names manually
eli5.show_weights(clf, vec=vectorizer, top=10, feature_names=feature_names)